# Apply trace metal concentrations to river classification

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import netCDF4 as nc
import datetime
from mpl_toolkits.basemap import Basemap, cm
import cmocean

%matplotlib inline

## River categories:

1. Glaciers
2. Continental
3. Other

In [2]:
rclass = nc.Dataset('/ocean/brogalla/GEOTRACES/data/river_class-202005.nc','r')
river_class = rclass.variables['rclass']
river_class = np.array(river_class)

### Apply trace metal concentrations

Notebook to find these concentratons: /analysis-brogalla/forcing/rivers/find_characteristic_tracer_concentrations.ipynb

In [14]:
pMn_rivers = np.zeros(river_class.shape)
SPM_rivers = np.zeros(river_class.shape)
dMn_rivers = np.zeros(river_class.shape)
pPb_rivers = np.zeros(river_class.shape)
dPb_rivers = np.zeros(river_class.shape)

In [15]:
multiplication_factor_glacial     = 1
multiplication_factor_continental = 1

SPM_factor = 0.35

Manganese:

In [16]:
dMn_rivers[river_class==1.0] = 164*multiplication_factor_glacial # glaciers
dMn_rivers[river_class==2.0] = 30*multiplication_factor_continental  # continental
dMn_rivers[river_class==3.0] = 5   # other
dMn_rivers[river_class==4.0] = 5   # Gulf of Boothia

In [17]:
SPM_rivers[river_class==1.0] = SPM_factor*261e-6*multiplication_factor_glacial # glaciers
SPM_rivers[river_class==2.0] = SPM_factor*1.196e-5*multiplication_factor_continental  # continental
SPM_rivers[river_class==3.0] = SPM_factor*3.372e-6   # other
SPM_rivers[river_class==4.0] = SPM_factor*3.372e-6   # Gulf of Boothia

In [18]:
pMn_rivers[river_class==1.0] = 151*multiplication_factor_glacial # glaciers
pMn_rivers[river_class==2.0] = 13*multiplication_factor_continental  # continental
pMn_rivers[river_class==3.0] = 2   # other
pMn_rivers[river_class==4.0] = 2   # Gulf of Boothia

Lead:

In [19]:
pPb_rivers[river_class==1.0] = 5*multiplication_factor_glacial # glaciers
pPb_rivers[river_class==2.0] = 1*multiplication_factor_continental  # continental
pPb_rivers[river_class==3.0] = 0.2   # other
pPb_rivers[river_class==4.0] = 0.2   # other

In [20]:
dPb_rivers[river_class==1.0] = 35*multiplication_factor_glacial # glaciers
dPb_rivers[river_class==2.0] = 43*multiplication_factor_continental  # continental
dPb_rivers[river_class==3.0] = 2   # other
dPb_rivers[river_class==4.0] = 2   # other

#### Convert units to mol/L:

In [21]:
dMn_rivers = dMn_rivers*10**(-9)      # nM --> 10^-3 mol/L
pMn_rivers = pMn_rivers*(1/54.938049)*10**(-6) # micro g/L --> 1.82*10^-2 mol/L

In [22]:
print(np.amax(dMn_rivers), np.amin(dMn_rivers), np.average(dMn_rivers))
print(np.amax(pMn_rivers))

1.64e-07 0.0 2.0868923611111135e-10
2.748550462722111e-06


In [23]:
dPb_rivers = dPb_rivers*10**(-12)           # pM --> 10^-3 mol/L
pPb_rivers = pPb_rivers*(1/207.2)*10**(-6) # micro g/L --> 1.82*10^-2 mol/L

In [24]:
print(np.amax(dPb_rivers))
print(np.amax(pPb_rivers))

4.3e-11
2.413127413127413e-08


##### Write to NetCDF file

In [25]:
c = nc.Dataset('/ocean/brogalla/GEOTRACES/data/runoff/ANHA12_runoff_monthly_combined_Dai_Trenberth_Bamber_y2002.nc','r')
lon_rf = np.array(c.variables['nav_lon'])
lat_rf = np.array(c.variables['nav_lat'])

In [26]:
ncd = nc.Dataset('/ocean/brogalla/GEOTRACES/data/river-forcing-best-guess-20201113.nc', 'w', zlib=True)
ncd.createDimension('x',len(c.dimensions['x']))
ncd.createDimension('y',len(c.dimensions['y']))

<class 'netCDF4._netCDF4.Dimension'>: name = 'y', size = 2400

In [27]:
ncd = nc.Dataset('/ocean/brogalla/GEOTRACES/data/river-forcing-best-guess-20201113.nc', 'w', zlib=True)
ncd.createDimension('x',len(c.dimensions['x']))
ncd.createDimension('y',len(c.dimensions['y']))

# variables
nav_lat = ncd.createVariable('nav_lat', 'float32', ('y','x'))
nav_lat.long_name = 'Latitude'
nav_lat.units = 'degrees_north'
nav_lat[:] = lat_rf

nav_lon = ncd.createVariable('nav_lon', 'float32', ('y','x'))
nav_lon.long_name = 'Longitude'
nav_lon.units = 'degrees_east'
nav_lon[:] = lon_rf

spm_rivers = ncd.createVariable('pmn_rivers', 'float32', ('y','x'))
spm_rivers.units = 'kg/L'
spm_rivers.long_name = 'Suspended Particulate Matter content'  
spm_rivers.coordinates = 'nav_lon nav_lat'
spm_rivers[:] = SPM_rivers

dmn_rivers = ncd.createVariable('dmn_rivers', 'float32', ('y','x'))
dmn_rivers.units = 'mol/L'
dmn_rivers.long_name = 'Dissolved Mn concentration'  
dmn_rivers.coordinates = 'nav_lon nav_lat'
dmn_rivers[:] = dMn_rivers

ppb_rivers = ncd.createVariable('ppb_rivers', 'float32', ('y','x'))
ppb_rivers.units = 'mol/L'
ppb_rivers.long_name = 'Particulate Pb concentration'  
ppb_rivers.coordinates = 'nav_lon nav_lat'
ppb_rivers[:] = pPb_rivers

dpb_rivers = ncd.createVariable('dpb_rivers', 'float32', ('y','x'))
dpb_rivers.units = 'mol/L'
dpb_rivers.long_name = 'Dissolved Pb concentration'  
dpb_rivers.coordinates = 'nav_lon nav_lat'
dpb_rivers[:] = dPb_rivers

ncd.close()